In [ ]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/FF++/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")


start_epoch = 24
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f" Resumed training from epoch {start_epoch}")
else:
    print(" No checkpoint found. Starting fresh training.")
   torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth'))


# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture
class HybridDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.3),
            nn.Linear(128, 64),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        combined_features = torch.cat([eff_features, vit_features], dim=1)
        return self.classifier(combined_features)

# Initialize model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridDeepfakeDetector().to(DEVICE)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Training Loop
EPOCHS = 30
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(True):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

# Model Evaluation

def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Single Image Prediction
def predict(image_path, model):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = val_transforms(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()
    return "Fake" if prediction > 0.5 else "Real"

# Example usage
print(predict("/content/testimage deepfake.jpg", model))


IndentationError: unindent does not match any outer indentation level (<string>, line 32)

In [1]:
import torch
import torch.nn as nn
import timm
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torchvision.datasets import ImageFolder
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
import os

# Set dataset path
dataset_path = "/Deepfake/code/FF++/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Device setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Architecture
class HybridDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Linear(2048, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        combined_features = torch.cat([eff_features, vit_features], dim=1)
        return self.classifier(combined_features)

# Initialize model, loss, optimizer
model = HybridDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scaler = torch.cuda.amp.GradScaler()

# Checkpoint resume logic
checkpoint_path = "checkpoint_epoch_19.pth"  # update if using different file
start_epoch = 20
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

# Transforms
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and prepare datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Training Loop with Checkpointing
EPOCHS = 30
for epoch in range(start_epoch, EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(True):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()

    avg_loss = train_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_loss:.4f}")
    scheduler.step()

    # ✅ Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

# Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Single image prediction
def predict(image_path, model):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = val_transforms(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()
    return "Fake" if prediction > 0.5 else "Real"

# Example usage
print(predict("/content/testimage_deepfake.jpg", model))


C:\Users\A\AppData\Local\Temp\ipykernel_20944\1655890072.py:52: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Resumed training from epoch 20
Training samples: 2012
Validation samples: 504
Test samples: 3552


C:\Users\A\AppData\Local\Temp\ipykernel_20944\1655890072.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 21/30 - Train Loss: 0.1868
Epoch 22/30 - Train Loss: 0.1872
Epoch 23/30 - Train Loss: 0.1891
Epoch 24/30 - Train Loss: 0.1955
Epoch 25/30 - Train Loss: 0.1939
Epoch 26/30 - Train Loss: 0.1916
Epoch 27/30 - Train Loss: 0.1843
Epoch 28/30 - Train Loss: 0.1848
Epoch 29/30 - Train Loss: 0.1943
Epoch 30/30 - Train Loss: 0.1918
Test Accuracy: 0.9296
Test AUC: 1.0000
Test F1 Score: 0.9167
Test Precision: 0.8462
Test Recall: 1.0000


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\content\\testimage_deepfake.jpg'